### Example usage for calculate_metrics

In [ ]:
import pickle
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils.audio_to_spectrograms import create_spectrogram_pkl
#from src.main import calculate_metrics

# Copied from src.main as I'm having problems with audio mamba imports
from src.config import CLASSES
from loguru import logger
import sed_eval

def calculate_metrics(pred_event_dict, gt_event_dict):
    logger.warning("Metrics not implemented!")
    # Based on event dictionary

    # Accuracy

    # IoU

    # DCASE SED eval: https://tut-arg.github.io/sed_eval/tutorial.html#id1
    event_based_metrics = sed_eval.sound_event.EventBasedMetrics(CLASSES)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        event_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(event_based_metrics)

In [3]:
# refer to single_stage_yamnet_frame.ipynb (Section "Get test set event predictions") for pickle file generation
pred_event_dict = pickle.load(open('outputs/single_stage_yamnet.pkl', 'rb'))

In [7]:
pred_event_dict

{'test_scene_0000.wav': [{'file': 'test_scene_0000.wav',
   'event_onset': 1.0,
   'event_offset': 1.5,
   'event_label': 'car_horn'},
  {'file': 'test_scene_0000.wav',
   'event_onset': 1.5,
   'event_offset': 3.0,
   'event_label': 'siren'},
  {'file': 'test_scene_0000.wav',
   'event_onset': 3.0,
   'event_offset': 4.5,
   'event_label': 'car_horn'}],
 'test_scene_0001.wav': [{'file': 'test_scene_0001.wav',
   'event_onset': 7.5,
   'event_offset': 9.0,
   'event_label': 'cough'}],
 'test_scene_0002.wav': [{'file': 'test_scene_0002.wav',
   'event_onset': 2.5,
   'event_offset': 6.0,
   'event_label': 'cough'}],
 'test_scene_0003.wav': [{'file': 'test_scene_0003.wav',
   'event_onset': 4.5,
   'event_offset': 6.0,
   'event_label': 'gun_shot'}],
 'test_scene_0004.wav': [{'file': 'test_scene_0004.wav',
   'event_onset': 6.5,
   'event_offset': 10.0,
   'event_label': 'car_horn'}],
 'test_scene_0005.wav': [{'file': 'test_scene_0005.wav',
   'event_onset': 6.5,
   'event_offset': 10.0,

In [ ]:
# check if pkl file is created
gt_pkl_path = 'data/processed/yamnet/spectrograms_test_list.pkl'
if not(os.path.exists(gt_pkl_path)):
    create_spectrogram_pkl()

In [ ]:
gt_events = pickle.load(open(gt_pkl_path, 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':ref_event['event_label']}]
                    for ref_event in gt_events}

In [8]:
gt_event_dict

{'test_scene_0000.wav': [{'file': 'test_scene_0000.wav',
   'event_onset': np.float64(1.4041677639819286),
   'event_offset': np.float64(4.726273878383591),
   'event_label': 'siren'}],
 'test_scene_0001.wav': [{'file': 'test_scene_0001.wav',
   'event_onset': np.float64(7.299327527971845),
   'event_offset': np.float64(8.555965450882997),
   'event_label': 'cough'}],
 'test_scene_0002.wav': [{'file': 'test_scene_0002.wav',
   'event_onset': np.float64(3.2337975402726475),
   'event_offset': np.float64(5.726486423359285),
   'event_label': 'cough'}],
 'test_scene_0003.wav': [{'file': 'test_scene_0003.wav',
   'event_onset': np.float64(4.000225463746228),
   'event_offset': np.float64(5.53271834268076),
   'event_label': 'gun_shot'}],
 'test_scene_0004.wav': [{'file': 'test_scene_0004.wav',
   'event_onset': np.float64(6.690817614225872),
   'event_offset': np.float64(10.0),
   'event_label': 'car_horn'}],
 'test_scene_0005.wav': [{'file': 'test_scene_0005.wav',
   'event_onset': np.flo

In [19]:
calculate_metrics(pred_event_dict, gt_event_dict)

2025-12-01 01:34:49.543 | WARNING  | __main__:calculate_metrics:14 - Metrics not implemented!


Event based metrics (onset-offset)
  Evaluated length                  : 3340.71 sec
  Evaluated files                   : 500 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 200.00 ms
  Offset (length)                   : 50.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 13.87 %
    Precision                       : 10.92 %
    Recall                          : 19.00 %
  Error rate
    Error rate (ER)                 : 2.34 
    Substitution rate               : 0.02 
    Deletion rate                   : 0.79 
    Insertion rate                  : 1.53 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 13.55 %
    Precision                       : 10.70 %
    Recall                          : 18.56 %
  Error rate
    Error rate (ER)                 : 2.36 
    Deletion rate                   : 0.81 
    Insertion rate 